# Pathology prediction (pneumothorax)

@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the pneumothorax pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples :  0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value. 

The model is based on binary classification for each target chest pathology.

Final sample size for the Pneumothorax pathology is : (N = 17,159)


#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.PNEUMOTHORAX, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Pneumothorax", experiment_name="CP_Pneumothorax", filepath="./results/pneumothorax")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='kfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-28 17:23:43,866	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=831922) Outer fold 1
(run_fold pid=831922) Configuring PyCaret for outer fold 1
(run_fold pid=831930) Outer fold 2


Processing:  75%|███████▌  | 3/4 [07:07<02:38, 158.29s/it] [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(run_fold pid=831922)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=831922) Fold                                                          
(run_fold pid=831922) 0       0.9117  0.8778  0.5508  0.9274  0.6911  0.6434  0.6732
(run_fold pid=831922) 1       0.9121  0.8681  0.5319  0.8889  0.6655  0.6186  0.6456
(run_fold pid=831922) 2       0.9021  0.8506  0.4796  0.8800  0.6208  0.5701  0.6048
(run_fold pid=831922) 3       0.9053  0.8689  0.5357  0.8898  0.6688  0.6175  0.6446
(run_fold pid=831922) 4       0.9080  0.8939  0.5270  0.9193  0.6699  0.6210  0.6535
(run_fold pid=831922) Mean    0.9078  0.8719  0.5250  0.9011  0.6632  0.6141  0.6444
(run_fold pid=831922) Std     0.0038  0.0141  0.0241  0.0187  0.0230  0.0240  0.0223
(run_fold pid=831930) Configuring PyCaret for outer fold 2
(run_fold pid=831922) Tuning hyperparameters for model xgboost with custom grid using grid search


                                                          
Processing:  75%|███████▌  | 3/4 [07:08<02:38, 158.67s/it]
                                                          


...

                                                          
Processing:  75%|███████▌  | 3/4 [06:59<02:35, 155.42s/it]
                                                          


(run_fold pid=831930) 0       0.9076  0.8793  0.5316  0.9211  0.6742  0.6248  0.6569
(run_fold pid=831930) 1       0.9112  0.8926  0.5209  0.8905  0.6573  0.6103  0.6393
(run_fold pid=831930) 2       0.9135  0.8676  0.5227  0.8932  0.6595  0.6138  0.6427
(run_fold pid=831930) 3       0.9044  0.8886  0.5432  0.8980  0.6769  0.6248  0.6520
(run_fold pid=831930) 4       0.9039  0.8619  0.5352  0.8613  0.6602  0.6078  0.6312
(run_fold pid=831930)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=831930) Fold                                                          
(run_fold pid=831930) Mean    0.9081  0.8780  0.5307  0.8928  0.6656  0.6163  0.6444
(run_fold pid=831930) Std     0.0037  0.0118  0.0082  0.0191  0.0082  0.0072  0.0091
(run_fold pid=831930) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=831922) Transformation Pipeline and Model Successfully Saved
(run_fold pid=831922)                        Model  Accuracy  

Processing:  75%|███████▌  | 3/4 [06:59<02:35, 155.28s/it]


(run_fold pid=831922)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=831922) Fold                                                          
(run_fold pid=831922) 0       0.9131  0.8854  0.5521  0.9177  0.6894  0.6425  0.6706
(run_fold pid=831922) 1       0.9071  0.8876  0.5515  0.8770  0.6772  0.6262  0.6493
(run_fold pid=831922) 2       0.9180  0.8816  0.5320  0.9059  0.6703  0.6271  0.6561
(run_fold pid=831922) 3       0.8994  0.8727  0.5224  0.8787  0.6552  0.6007  0.6286
(run_fold pid=831922) 4       0.9053  0.8914  0.5242  0.9075  0.6645  0.6139  0.6453
(run_fold pid=831922) Mean    0.9086  0.8838  0.5364  0.8974  0.6713  0.6221  0.6500
(run_fold pid=831922) Std     0.0064  0.0063  0.0130  0.0165  0.0116  0.0140  0.0137
(run_fold pid=831922) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=831930) Transformation Pipeline and Model Successfully Saved
(run_fold pid=831930)                        Model  Accuracy     AUC  ...      F1   Kappa    MCC
(run_fold pid=831930) 0  Extreme Gradient Boosting    0.9237  0.9278  ...  0.7364  0.6946  0.721
(run_fold pid=831930) 
(run_fold pid=831930) [1 rows x 8 columns]
(run_fold pid=831922) Transformation Pipeline and Model Successfully Saved
(run_fold pid=831922)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=831922) 0  Extreme Gradient Boosting    0.9166  0.9093  ...  0.7058  0.6598  0.6814
(run_fold pid=831922) 
(run_fold pid=831922) [1 rows x 8 columns]
Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.91096  0.004770
1       AUC  0.89616  0.006729
2    Recall  0.54564  0.020857
3     Prec.  0.92914  0.009705
4        F1  0.68742  0.018629
5     Kappa  0.63956  0.020556
6       MCC  0.67046  0.018303
